In [2]:
from google.colab import drive
drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
import sys
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

from scipy.io import wavfile
import librosa
from tqdm import tqdm

from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import librosa
import csv
import torch
from torch.utils import data
if 'google_drive_downloader' not in sys.modules:
    get_ipython().system('pip install googledrivedownloader')
from google_drive_downloader import GoogleDriveDownloader as gdd

In [0]:
dataset_path = os.path.join(gdrive_root, 'dataset')
if not os.path.exists(dataset_path):
  os.makedirs(dataset_path)
train_tar_path = os.path.join(dataset_path, "train.tar.gz")
valid_tar_path = os.path.join(dataset_path, "valid.tar.gz")
test_tar_path = os.path.join(dataset_path, "test.tar.gz")

In [5]:
# Download Train set
if not os.path.exists(train_tar_path):

  gdd.download_file_from_google_drive(file_id='1CULVCAq0T3wqZTPGIqPja6OtwjYJkGAy',
                                      dest_path=train_tar_path,
                                      unzip=False, showsize=True)

19.7 MiB 

KeyboardInterrupt: ignored

In [0]:
%cd /gdrive/My\ Drive/dataset
!tar xvzf /gdrive/My\ Drive/dataset/train.tar.gz

In [6]:
# Download Valid set
if not os.path.exists(valid_tar_path):

  gdd.download_file_from_google_drive(file_id='1WE229Jt9WV2iZbxY7YjkYIfSZyCHz9Iq',
                                    dest_path=valid_tar_path,
                                    unzip=False, showsize=True)

1.4 GiB tar (child): dataset/valid.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [13]:
%cd /gdrive/My\ Drive/dataset
!tar xvzf valid.tar.gz

/gdrive/My Drive/dataset
valid_clean/
valid_clean/valid_clean2385.wav
valid_clean/valid_clean2208.wav
valid_clean/valid_clean2421.wav
valid_clean/valid_clean3840.wav
valid_clean/valid_clean3906.wav
valid_clean/valid_clean558.wav
valid_clean/valid_clean1738.wav
valid_clean/valid_clean1158.wav
valid_clean/valid_clean1960.wav
valid_clean/valid_clean123.wav
valid_clean/valid_clean751.wav
valid_clean/valid_clean268.wav
valid_clean/valid_clean449.wav
valid_clean/valid_clean426.wav
valid_clean/valid_clean3647.wav
valid_clean/valid_clean1973.wav
valid_clean/valid_clean2750.wav
valid_clean/valid_clean193.wav
valid_clean/valid_clean2565.wav
valid_clean/valid_clean1491.wav
valid_clean/valid_clean2096.wav
valid_clean/valid_clean549.wav
valid_clean/valid_clean2732.wav
valid_clean/valid_clean310.wav
valid_clean/valid_clean1995.wav
valid_clean/valid_clean2698.wav
valid_clean/valid_clean6.wav
valid_clean/valid_clean2013.wav
valid_clean/valid_clean845.wav
valid_clean/valid_clean648.wav
valid_clean/vali

In [0]:
# Download Test set
if not os.path.exists(test_tar_path):

  gdd.download_file_from_google_drive(file_id='1mERBbcwBgGjRpHeGf3d871DEW4EvI7fi',
                                      dest_path=test_tar_path,
                                      unzip=False, showsize=True)

In [0]:
%cd /gdrive/My\ Drive/dataset
!tar xvzf /gdrive/My\ Drive/dataset/test.tar.gz

In [15]:
# Data Loader Part

# DATA LOADING - LOAD FILE LISTS
def load_data_list(folder=dataset_path, setname='train'):
    assert(setname in ['train', 'valid'])

    dataset = {}
    foldername = folder + '/' + setname
    

    print("Loading files...")
    dataset['innames'] = []
    dataset['outnames'] = []
    dataset['shortnames'] = []

    noisy_filelist = os.listdir("%s_noisy"%(foldername))
    # filelist = [f for f in filelist if f.endswith(".wav")]
    for i in tqdm(noisy_filelist):
        dataset['innames'].append("%s_noisy/%s"%(foldername,i))
        dataset['shortnames'].append("%s"%(i))
        
    clean_filelist = os.listdir("%s_clean"%(foldername))
    for i in tqdm(clean_filelist):
        dataset['outnames'].append("%s_clean/%s"%(foldername,i))

    return dataset

# DATA LOADING - LOAD FILE DATA
def load_data(dataset):

    dataset['inaudio']  = [None]*len(dataset['innames'])
    dataset['outaudio'] = [None]*len(dataset['outnames'])

    for id in tqdm(range(len(dataset['innames']))):

        if dataset['inaudio'][id] is None:
            inputData, sr = librosa.load(dataset['innames'][id], sr=None)
            outputData, sr = librosa.load(dataset['outnames'][id], sr=None)

            shape = np.shape(inputData)

            dataset['inaudio'][id]  = np.float32(inputData)
            dataset['outaudio'][id] = np.float32(outputData)

    return dataset

class AudioDataset(data.Dataset):
    """
    Audio sample reader.
    """

    def __init__(self, data_type):
        dataset = load_data_list(setname=data_type)
        self.dataset = load_data(dataset)

        self.file_names = dataset['innames']

    def __getitem__(self, idx):
        mixed = torch.from_numpy(self.dataset['inaudio'][idx]).type(torch.FloatTensor)
        clean = torch.from_numpy(self.dataset['outaudio'][idx]).type(torch.FloatTensor)

        return mixed, clean

    def __len__(self):
        return len(self.file_names)

    def zero_pad_concat(self, inputs):
        max_t = max(inp.shape[0] for inp in inputs)
        shape = (len(inputs), max_t)
        input_mat = np.zeros(shape, dtype=np.float32)
        for e, inp in enumerate(inputs):
            input_mat[e, :inp.shape[0]] = inp
        return input_mat

    def collate(self, inputs):
        mixeds, cleans = zip(*inputs)
        seq_lens = torch.IntTensor([i.shape[0] for i in mixeds])

        x = torch.FloatTensor(self.zero_pad_concat(mixeds))
        y = torch.FloatTensor(self.zero_pad_concat(cleans))

        batch = [x, y, seq_lens]
        return batch

# Below is how to use data loader

train_dataset = AudioDataset(data_type='train')
train_data_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
       collate_fn=train_dataset.collate, shuffle=True, num_workers=4)
valid_dataset = AudioDataset(data_type='valid')
valid_data_loader = DataLoader(dataset=valid_dataset, batch_size=4,
        collate_fn=valid_dataset.collate, shuffle=False, num_workers=4)

  0%|          | 0/4000 [00:00<?, ?it/s]

Loading files...


100%|██████████| 4000/4000 [45:48<00:00,  1.18s/it]


[tensor([[ 0.0071,  0.0004, -0.0042,  ...,  0.0164,  0.0220,  0.0246],
        [ 0.0523,  0.0585,  0.0588,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0030, -0.0136, -0.0170,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0362, -0.0003,  0.0238,  ...,  0.0000,  0.0000,  0.0000]]), tensor([[-0.0361, -0.0299, -0.0191,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0037, -0.0041, -0.0042,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0016,  0.0013,  0.0020,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0110,  0.0043, -0.0075,  ..., -0.0009, -0.0028, -0.0039]]), tensor([437856, 193088,  68016,  70880], dtype=torch.int32)]
3
torch.Size([4, 437856])
torch.Size([4, 126224])
torch.Size([4])
